# Knowledge For Agents

### Setup

In [79]:
# !pip install crewai
# !pip install crewai-tools


# Installing it to see traces(Found some issues with passing knowledge at crew level)
# !pip install langtrace-python-sdk

In [80]:
# ollama pull llama3.2:1b

### Imports

In [9]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [10]:
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

from crewai import Agent, Task, Crew, Process

In [11]:
from langtrace_python_sdk import langtrace
from langchain_openai import ChatOpenAI

# Define OpenAI LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7
)

langtrace.init(api_key = os.environ.get('LANGTRACE_API_KEY'))

Initializing Langtrace SDK..
⭐ Leave our github a star to stay on top of our updates - https://github.com/Scale3-Labs/langtrace
Exporting spans to Gaurav Kandel..
Langtrace Project URL: https://app.langtrace.ai/project/cmbib9ry1002pbk2m5r68eixs/traces


### Define Knowledge For Agent

In [12]:
policy_text = """
Our return policy allows customers to return
any product within 30 days of purchase. Refunds
will be issued only if the item is unused and in original packaging.
Customers must provide proof of purchase when requesting a return.
"""

# Create a StringKnowledgeSource object
return_policy_knowledge = StringKnowledgeSource(content=policy_text)

### Define Agent, Task & Crew

In [13]:
returns_agent = Agent(
    role = "Product Returns Assistant",
    goal = "Answer customer questions about return policy accurately.",
    backstory = """You work in customer service and specialize
                 in returns, refunds, and policies.""",
    allow_delegation = False,
    # knowledge_sources = [return_policy_knowledge],
    verbose = True,
    llm = "gpt-4o",
    # llm=llm,
    # callback_manager = langtrace
)

In [14]:
returns_task = Task(
    description = "Answer the customer question: {question}",
    expected_output = "A concise and accurate answer.",
    agent = returns_agent
)

In [15]:
crew = Crew(
    agents = [returns_agent],
    tasks = [returns_task],
    process = Process.sequential,
    knowledge_sources = [return_policy_knowledge],
    verbose = True,
    # callback_manager = langtrace
)

 
[2025-06-05 01:06:20][ERROR]: Failed to upsert documents: ValidationError.__new__() missing 1 required positional argument: 'line_errors'
 
[2025-06-05 01:06:20][WARNING]: Failed to init knowledge: ValidationError.__new__() missing 1 required positional argument: 'line_errors'


In [16]:
query = "Can I get a refund if I used the item once?"

result = crew.kickoff(inputs = {"question": query})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0ad84af9-a9f8-483a-a413-9a1d1c844021                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b6a995b8-605c-44a2-bb37-89fd1748690c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b6a995b8-605c-44a2-bb37-89fd1748690c
       Status: Executing Task...
    └── 🤖 Agent: Product Returns Assistant
            Status: In Progress

# Agent: Product Returns Assistant
## Task: Answer the customer question: Can I get a refund if I used the item once?


# Agent: Product Returns Assistant
## Final Answer: 
Whether or not you can receive a refund for an item you have used once largely depends on the specific return policy of the store or company from which you purchased the item. Typically, many retailers have a return policy that allows for returns and refunds within a certain time frame even if the item has been used, as long as it is in its original condition and packaging. However, some items, particularly those involving personal use and hygiene, may not be eligible for return once used. For a definitive answer, it's best to refer to the specific return policy of the retailer or contact their customer service directly for clarification.




🚀 Crew: crew
└── 📋 Task: b6a995b8-605c-44a2-bb37-89fd1748690c
       Status: Executing Task...
    └── 🤖 Agent: Product Returns Assistant
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: b6a995b8-605c-44a2-bb37-89fd1748690c
       Assigned to: Product Returns Assistant
       Status: ✅ Completed
    └── 🤖 Agent: Product Returns Assistant
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b6a995b8-605c-44a2-bb37-89fd1748690c                                                                     │
│  Agent: Product Returns Assistant                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ad84af9-a9f8-483a-a413-9a1d1c844021                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [89]:
crew.knowledge

In [90]:
returns_agent.knowledge

Knowledge(sources=[StringKnowledgeSource(chunk_size=4000, chunk_overlap=200, chunks=[], chunk_embeddings=[], storage=None, metadata={}, collection_name=None, content='\nOur return policy allows customers to return\nany product within 30 days of purchase. Refunds\nwill be issued only if the item is unused and in original packaging.\nCustomers must provide proof of purchase when requesting a return.\n')], storage=<crewai.knowledge.storage.knowledge_storage.KnowledgeStorage object at 0x166021990>, embedder=None, collection_name=None)

In [91]:
from IPython.display import Markdown
Markdown(result.raw)

Whether you can receive a refund for an item you've used once depends on our return policy. Generally, if the item is in a resalable condition and returned within the specified return period with the original receipt, you may be eligible for a refund. However, some items may have specific restrictions or exceptions. Please check our detailed return policy or contact customer service for product-specific return information.